# Cross-Datasite Query Example: Bob Accessing Alice's Data

This notebook demonstrates how Bob can query Alice's data using remote LLM functionality. It assumes:
- Both Bob and Alice have already run their respective setup notebooks
- Alice has granted Bob permission to access specific files
- Bob now wants to analyze Alice's data without having direct access to it

In [1]:
from llama_remote import client

## 1. Connect as Bob

In [2]:
# Creating clients
bob_client = client("~/.syft_bob_config.json")

# for development purposes, also connect as alice in case you're changing the api. 
alice_client = client("~/.syft_alice_config.json")

In [4]:
# Re-request the permissions list
permitted_files = bob_client.list_permitted_files(
    to_email="alice@openmined.org",
    user_email=bob_client.client.email
)

# Print the files Bob has permission to access on Alice's datasite
print("Files Bob can access on Alice's datasite:")
for file_path in permitted_files.allowed_files:
    print(f"  - {file_path}")

Files Bob can access on Alice's datasite:
  - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv
  - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/public/iphone_2007.txt


In [7]:
# Get the patient data file path from the permitted files
patient_data_file = next((f for f in permitted_files.allowed_files if "patient_data.csv" in f), None)

In [6]:
# Ask a specific question about treatment effectiveness
if patient_data_file:
    prompt = "Based on the patient data, which treatment appears to be most effective? What evidence supports this conclusion?"
    
    print("Asking about treatment effectiveness...")
    response = bob_client.generate(
        to_email="alice@openmined.org",
        model="llama3",
        prompt=prompt,
        system="You are a medical researcher. Analyze the data and provide evidence-based conclusions.",
        temperature=0.7,
        files=[patient_data_file]
    )
    
    print("\nResponse about treatment effectiveness:")
    print("-" * 80)
    print(response.response)
    print("-" * 80)

Asking about treatment effectiveness...

Response about treatment effectiveness:
--------------------------------------------------------------------------------
After analyzing the patient data, I conclude that Therapy C is the most effective treatment based on the outcome "Significantly Improved". This conclusion is supported by the following evidence:

1. **Frequency of improvement**: Two patients (P001 and P005) received Therapy C, and both had an improved outcome, which suggests a higher rate of success compared to other treatments.
2. **Consistency of outcome**: The outcome for patients receiving Therapy C was consistent, with both patients experiencing significant improvement. This consistency implies that the treatment is effective in achieving its intended goal.
3. **Diverse patient population**: The patients who received Therapy C (P001 and P005) had different ages (29 and 45) and genders (F), which suggests that the treatment may be effective across a broad range of patients

In [8]:
# Try to access Alice's research notes (assuming Bob doesn't have permission)
research_notes_path = patient_data_file.replace("patient_data.csv", "research_notes.txt")

print(f"Attempting to access unauthorized file: {research_notes_path}")
response = bob_client.generate(
    to_email="alice@openmined.org",
    model="llama3",
    prompt="Summarize the research notes and main findings.",
    system="You are a research assistant. Provide a concise summary of the research.",
    temperature=0.7,
    files=[research_notes_path]
)

print("\nResponse when trying to access unauthorized file:")
print("-" * 80)
print(response.response)
print("-" * 80)

Attempting to access unauthorized file: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/research_notes.txt

Response when trying to access unauthorized file:
--------------------------------------------------------------------------------
Error: User bob@openmined.org does not have permission to access file: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/research_notes.txt
--------------------------------------------------------------------------------
